# Part 5 - Product Score
In this notebook, we will calculate a product score for each category's product, to determine determine the 3 best ones, or the worst one.

## Imports

In [2]:
import os
import pandas as pd
from tqdm import tqdm
tqdm.pandas()  # Initialize tqdm with pandas

# import helpers module
import helpers
import importlib
importlib.reload(helpers)

SEP = 100 * '-'

## Load datasets

In [3]:
from functools import reduce

# Load datasets
data_processed = helpers.load_pickled_dataset('pickle/data_processed.pkl')
data_clustered = helpers.load_pickled_dataset('pickle/data_clustered.pkl')
data_sa = helpers.load_pickled_dataset('pickle/data_sentiment_analysis.pkl')

# Select columns
proc_columns = ['name', 'brand', 'reviews.rating', 'reviews.numHelpful', 'review']
cluster_columns = ['review', 'clusterCategories']
sa_columns = ['review', 'reviews.sentiment', 'reviews.ft', 'reviews.ft.sentiment']

# Filter DataFrames by the selected columns
data1 = data_processed[proc_columns]
data2 = data_clustered[cluster_columns]
data3 = data_sa[sa_columns]

# Merge the DataFrames sequentially
data = reduce(lambda left, right: pd.merge(left, right, on='review', how='outer'), [data1, data2, data3])


Dataset loaded from pickle/data_processed.pkl.
Dataset loaded from pickle/data_clustered.pkl.
Dataset loaded from pickle/data_sentiment_analysis.pkl.


In [4]:
helpers.print_random_product_sheet(data)

----------------------------------------------------------------------------------------------------
[name] Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes Special Offers, Magenta
----------------------------------------------------------------------------------------------------
[brand] Amazon
----------------------------------------------------------------------------------------------------
[reviews.rating] 5.0
----------------------------------------------------------------------------------------------------
[reviews.numHelpful] 0.0
----------------------------------------------------------------------------------------------------
[review] Great for the Kids
Kids love it! We are Prime members, good kid friendly case... purchased a 128gb memory card to hold all kinds of games and movies
I recommend this product.
----------------------------------------------------------------------------------------------------
[clusterCategories] Home & Smart Devices
------------------------------

## Split products per category

In [ ]:
# create a new dataframe to store the products for each category
data_scored = pd.DataFrame(columns=['clusterCategories', 'name', 'score'])

# iterage over the dataset
for index, row in data.iterrows():
    category = row['clusterCategories']
    name = row['name']
    rating = row['reviews.rating']

    # Check if the product already exists in the new dataframe
    if (data_scored['name'] == name).any():
        # if it exists, update the score
        data_scored.loc[data_scored['name'] == name, 'score'] += rating
    else:
        # If the product is not in the dataframe, add it as a new row
        new_row = pd.DataFrame({'clusterCategories': [category], 'name': [name], 'score': [rating]})
        data_scored = pd.concat([data_scored, new_row], ignore_index=True)


In [35]:
import pandas as pd

def calculate_scores(data):
    # Step 1: Group by category and name and sum the ratings for each product
    grouped_data = data.groupby(['clusterCategories', 'name'])['reviews.rating'].sum().reset_index()

    # Step 2: Sort by category and score (descending)
    sorted_data = grouped_data.sort_values(by=['clusterCategories', 'reviews.rating'], ascending=[True, False])

    # Step 3: Select the top 3 products per category
    # top_products_df = sorted_data.groupby('category', group_keys=False).apply(lambda x: x.nlargest(3, 'score')).reset_index(drop=True)
    
    return sorted_data

# calculate scores and store them in a new dataframe
data_scored = calculate_scores(data)

# check if we have duplicates
duplicates = data_scored[data_scored.duplicated(subset=['clusterCategories', 'name'], keep=False)]
print("Duplicates:", not duplicates.empty)

print(data_scored.info())
data_scored

Duplicates: False
<class 'pandas.core.frame.DataFrame'>
Index: 145 entries, 7 to 141
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   clusterCategories  145 non-null    object 
 1   name               145 non-null    object 
 2   reviews.rating     145 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.5+ KB
None


,clusterCategories,name,reviews.rating
7,Accessories & Adapters,AmazonBasics AAA Performance Alkaline Batterie...,62201.0
6,Accessories & Adapters,AmazonBasics AA Performance Alkaline Batteries...,43195.0
19,Accessories & Adapters,"Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16...",40862.0
18,Accessories & Adapters,"Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16...",25108.0
20,Accessories & Adapters,"Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16...",23945.0
...,...,...,...
142,Video & Streaming,Certified Refurbished Amazon Fire TV with Alex...,56.0
144,Video & Streaming,Fire TV Stick Streaming Media Player Pair Kit,30.0
140,Video & Streaming,Amazon Fire TV with 4K Ultra HD and Alexa Voic...,20.0
139,Video & Streaming,Amazon Fire TV Gaming Edition Streaming Media ...,14.0


## Extract the 3 best products per category

In [39]:
# print unique categories
category_names = data_scored['clusterCategories'].unique()
for cat in category_names:
    print(cat)

Accessories & Adapters
Electronics & Media
Home & Smart Devices
Pet Supplies
Tablets & eReaders
Video & Streaming


In [47]:
# Select the top 3 products per category
def best_products_per_category(data, category_name, top_n=3):
    # subset the category
    category_subset = data_scored[data_scored['clusterCategories'] == category_name]
    # select the top n products
    return category_subset.head(top_n)

best_products_per_category(data_scored, cat, 3)


,clusterCategories,name,reviews.rating
143,Video & Streaming,"Echo (White),,,\r\nEcho (White),,,",67.0
142,Video & Streaming,Certified Refurbished Amazon Fire TV with Alex...,56.0
144,Video & Streaming,Fire TV Stick Streaming Media Player Pair Kit,30.0


In [46]:
def get_worst_products_per_category(data, category_name, bottom_n=1):
    # subset the category
    category_subset = data_scored[data_scored['clusterCategories'] == category_name]
    # select the bottom n products
    return category_subset.tail(bottom_n)

get_worst_products_per_category(data_scored, cat, 1)

,clusterCategories,name,reviews.rating
141,Video & Streaming,Brand New Amazon Kindle Fire 16gb 7 Ips Displa...,5.0


## Pickle scored dataset

In [48]:
data_scored.to_pickle('pickle/data_scored.pkl')